In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import freqopttest.util as util
import freqopttest.data as data
import freqopttest.kernel as kernel
import freqopttest.tst as tst
import freqopttest.glo as glo
import sys

In [ ]:
# sample source 
m = 20000
dim = 200
seed = 8
n = m
ss = data.SSGaussMeanDiff(dim, my=0.5)
#ss = data.SSGaussVarDiff(dim)
#ss = data.SSSameGauss(dim)
#ss = data.SSBlobs()
dim = ss.dim()
tst_data = ss.sample(m, seed=seed)
tr, te = tst_data.split_tr_te(tr_proportion=0.5, seed=10)
#te = tst_data

## grid search to choose the best Gaussian width

In [ ]:
J = 5
alpha = 0.01

T = tst.MeanEmbeddingTest.init_locs_2randn(tr, J, seed=seed+1)
#T = np.random.randn(J, dim)

In [ ]:
#mean_sd = tr.mean_std()
#print('mean_sd: %.3g'%mean_sd)
#scales = 2.0**np.linspace(-3, 3, 20)
#list_gwidth = np.hstack( (mean_sd*scales*(dim**0.5), 2**np.linspace(-20, 10, 20) ))

med = util.meddistance(tr.stack_xy(), 1000)
list_gwidth = np.hstack( ( (med**2) *(2.0**np.linspace(-5, 5, 40) ) ) )
list_gwidth.sort()
besti, powers = tst.MeanEmbeddingTest.grid_search_gwidth(tr, T, list_gwidth, alpha)

# plot
plt.plot(list_gwidth, powers, 'o-')
plt.xscale('log', basex=2)
plt.xlabel('Gaussian width')
plt.ylabel('Test power')
plt.title('Median dist: %.3g. Best gwidth2**0.5: %.3g'%(med, list_gwidth[besti]**0.5) )

In [ ]:
# test with the best Gaussian with 
best_width = list_gwidth[besti]
met_grid = tst.MeanEmbeddingTest(T, best_width, alpha)
met_grid.perform_test(te)